In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import acf, pacf, graphics
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
import pmdarima as pm
plt.rcParams.update({'figure.figsize': (8,5)})

## Data Prepare

In [ ]:
# load data
df = pd.read_csv('data/project/SCM_TMS_PARTS_LOGS_NEW.csv')
# single part
df = df[df['PART_NO']=='85-EMA0900']
# transfer time to YYMM, and group by
df['STOCK_EVENT_TIME'] = pd.to_datetime(df['STOCK_EVENT_TIME']).dt.strftime('%Y-%m')
df['STOCK_EVENT_TIME'] = pd.to_datetime(df['STOCK_EVENT_TIME'])
df = df[['STOCK_EVENT_TIME','QTY']].groupby(['STOCK_EVENT_TIME']).sum().sort_values(by=['STOCK_EVENT_TIME'])

In [ ]:
df.plot()

## AR (Auto Regression)

In [ ]:
# 確認時間序列平穩性，透過 P-Value 來檢測 (p < 0.05)
#   為何需要求資料平穩性？變數的歷史和現狀呈現出的基本特性，在未來階段的一個長時期里會維持不變
#   如果p值小於 0.05表示序列不穩定，就會建議不使用AR模型
result = adfuller(df.QTY)
print('p-value: %.2f' % result[1])

In [ ]:
# 因 P-Value > 0.05，先測試一階差異再看分數
df_diff1 = df.QTY.diff()
result = adfuller(df_diff1.dropna())
print('p-value: %.2f' % result[1])

In [ ]:
# 自相關
autocorrelation_plot(df.QTY.tolist())

In [ ]:
# 自相關/偏自相關
#    自相關：考慮中間時刻值的影響, 例如t-3對t影響, 會同時考慮t-1, t-2對t的影響
#    偏自相關：不考慮中間時刻的影響
fig, axes = plt.subplots(2, 1)
plot_acf(df.values.squeeze(), ax=axes[0]) #自相關, 長拖尾，需做差分
plot_pacf(df.values.squeeze(), ax=axes[1]) #偏相關
plt.tight_layout()
plt.show()

In [ ]:
# STL Seasonal-Trend decomposition using LOESS
stl = STL(df['QTY'], seasonal=3)
res = stl.fit()
fig = res.plot()
# Multiplicative Decomposition (乘法)
result_mul = seasonal_decompose(df['QTY'], model='multiplicative', extrapolate_trend='freq')
# Additive Decomposition (加法)
result_add = seasonal_decompose(df['QTY'], model='additive', extrapolate_trend='freq')
# Plot
result_mul.plot().suptitle('Multiplicative Decompose')
result_add.plot().suptitle('Additive Decompose')
plt.show()

In [ ]:
mod = AutoReg(df.values.squeeze(), 3, old_names=False)
res = mod.fit()
print(res.summary())

In [ ]:
fig = res.plot_predict(0, 70)

## ARIMA (AR Integrated MA)

In [ ]:
# Data Prepare
df = pd.read_csv('data/project/SCM_TMS_PARTS_LOGS_NEW.csv')
df = df[(df['PART_NO']=='85-ECT1190')]
df['STOCK_EVENT_TIME'] = pd.to_datetime(df['STOCK_EVENT_TIME']).dt.strftime('%Y%m')
df = df[['STOCK_EVENT_TIME','QTY']].groupby(['STOCK_EVENT_TIME']).sum().sort_values(by=['STOCK_EVENT_TIME'])

In [ ]:
# Original Series
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(df); axes[0, 0].set_title('Original Series')
plot_acf(df, ax=axes[0, 1])
# 1st Differencing
axes[1, 0].plot(df.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(df.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(df.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(df.diff().diff().dropna(), ax=axes[2, 1])

plt.show()

In [ ]:
model = pm.auto_arima(df, start_p=1, start_q=1,
                      information_criterion='aic',
                      test='adf',       # use adftest to find optimal 'd'
                      #max_p=5, max_q=5, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
model = ARIMA(df, order=(4,0,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()
# Actual vs Fitted
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
df = df.reset_index(drop=True)
# Create Training and Test
train = df.loc[:62]
test = df.loc[62:]

# Build Model
# model = ARIMA(train, order=(3,,1))  
model = ARIMA(train, order=(4,0,2))  
fitted = model.fit(disp=0)  

# Forecast
fc, se, conf = fitted.forecast(3, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
print(test)
print(fc_series.round())

In [ ]:
_accsum=0
_acount=0
for index, row in test.iterrows():
    _act=row['QTY']
    _prd=fc_series.round().loc[index]
    if 1- abs((_prd - _act)/_act ) > 0 :
        _accsum += (1- abs((_prd - _act)/_act ))
    _acount += 1
    
#print(f'acc: {round(_accsum*100/test.shape[0],2)}')
print(f'acc: {round(_accsum*100/_acount,2)}')

# SARIMA

# SARIMAX 